## TEST

In [1]:
from Levenshtein import distance
import tensorflow as tf
import os

2024-05-09 10:01:17.447669: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 10:01:17.451329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 10:01:17.858563: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 10:01:18.700480: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 10:01:24.660346: W tensorflow/compiler/tf2

In [2]:
dataset_path = 'dataset/transfer_dataset/'
val_dataset_path = os.path.join(dataset_path, 'val')

In [3]:
from inference_smartapp import handwriting_model

models/keras/Model9v3_Words_Transfer
Loading pre-trained model and weights...


2024-05-09 10:01:39.870313: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 10:01:41.633537: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 10:01:41.633596: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 10:01:41.635820: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 10:01:41.635857: I external/local_xla/xla/stream_executor

Model and weights loaded successfully.


In [4]:
val_df_list = os.listdir(val_dataset_path)
val_df_jpg_list = [val_df_list[i] for i in range(len(val_df_list)) if val_df_list[i].endswith('.jpg')]
val_df_jpg_list = [os.path.join(val_dataset_path, val_df_jpg_list[i]) for i in range(len(val_df_jpg_list)) ]

In [5]:
scores = []
pred_real = []
for image_path in val_df_jpg_list:
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, 1)
    prediction = handwriting_model.inference(image)
    with open(image_path.replace('.jpg', '.txt'), 'r') as f:
        label = f.read().replace("|", " ")
    scores.append(distance(prediction, label))
    pred_real.append((prediction, label))
    #print(f'Prediction: {prediction} Label: {label} Distance: {scores[-1]}')


2024-05-09 10:01:48.863660: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


1/1 [==============================] - 1s 690ms/step


In [6]:
print(f'Mean distance: {round(sum(scores) / len(scores),2)}')

Mean distance: 0.48


In [8]:
print("(Prediction,Real)")
for pred_r in pred_real:
    print(pred_r)

(Prediction,Real)
('Lndtke', 'Lüdtke')
('Markus', 'Markus')
('3366806', '3366806')
('Iuedtkke@gmx.de', 'luedtke@gmx.de')
('Ludtke', 'Lüdtke')
('Mila', 'Mila')
('6b', '6b')
('Schumannstrasje 10', 'Schumannstrasse 10')
('28213', '28213')
('Bremen', 'Bremen')
('Schubert', 'Schubert')
('Leni', 'Leni')
('6B', '6B')
('Kiattenweg 2', 'Klattenweg 2')
('28213', '28213')
('Bremen', 'Bremen')
('Schubert', 'Schubert')
('Tomas', 'Thomas')
('5804474', '5804474')
('tchubot@gmx.de', 'tschubert@gmx.de')
('Finn', 'Finn')
('6B', '6B')
('Bremen', 'Bremen')
('28309', '28309')
('Marschsssase2', 'Marschstrasse 2')
('meischristiar@gmx.de', 'merschristian@gmx.de')
('43ua733', '4349733')
('Chriltian', 'Christian')
('Mersmonn', 'Mersmann')
('Mersmann', 'Mersmann')
('Heydrich', 'Heydrich')
('Anika', 'Anika')
('12A', '12A')
('Band', 'Band')
('Fussball', 'Fussball')
('Programmierung', 'Programmierung')
('Gride', 'Güde')
('Insa', 'Insa')
('121', '12A')
('Fussball', 'Fussball')
('Basketball', 'Basketball')
('Chor', '